In [90]:

import shutil
shutil.rmtree('__pycache__', ignore_errors=True)

from tqdm import tqdm
import time
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
import importlib
# Our functions
from EvalFunctions import AUCEval, MMREval, nDCGEval
from RecAlgs import MostPopBaseline, CollaborativeFiltering, Hybrid, News_Recommender_CBF

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The parameters to select how to tun the algorithm

In [91]:
# General
TimeCutOffDays = 3              # How old the articles can be that we would consider recommending (older than X days are not considered)
AmountToPredict = 10

# Data selection                
TrainingDataStartDate = 0       # From what day we want to collect data
TrainingDataWindowSize = 2      # How many days of training data we want, 0 for all
TestDataWindowSize = 1          # How many days of test data we want, 0 for all

# Algorithm specifics 
TypeOfRecAlg = 3                # Which RecAlg we want to use 0-Pop, 1-CBF, 2-CF, 3-Hybrid

# Popular Baseline
TimePenaltyPerHour = 0.1        # The percentage on penalty per hour the news gets
TimePenaltyStart = 24           # After howmany hours in the past the penalty starts

# Random Baseline
MinScore = 0                    # Minimum score that can be given
MaxScore = 1                    # Maximum score that can be given

# Content based filtering

# Collaborative filtering

# Hybrid
UsePopBaseline = False          # Whether to use Popularity baseline
UseRandBaseLine = False         # Whether to use Random baseline
UseCBF = True                   # Whether to use Content based filtering
UseCF = True                    # Whether to use Collaborative filtering
TakeMax = False                 # Whether to take the max between CBF and CF before applying weights
Weights = [0.2, 0.4, 0.4]       # The weights for the different parts (in order of appearance above)


Data selection

In [92]:
def getGroundTruth(FutureBehaviors, RequestedUserID, CurrentInstanceClickData):
    # Filter only relevant rows
    UserData = FutureBehaviors[FutureBehaviors['UserID'] == RequestedUserID]
    # Extract clicked articles
    ClickedArticles = []
    for clicks in UserData['ClickData']:
        ClickedArticles.extend(
            click.replace("-1", "") for click in clicks.split(" ") if click.endswith("-1")
        )
    ClickedArticles.extend(
        CurrentInstanceClickData.replace("-1", "") for aclick in CurrentInstanceClickData.split(" ") if aclick.endswith("-1")
    )
    return ClickedArticles



In [93]:
PreparationTime = 0
PreparationStartTime = time.time()

# Add the first time the article has been seen in the behaviors as the Est_PublishedTime in the articles.
AllTrainingData = pd.read_csv("../data/MINDsmall_train/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
AllValidationData = pd.read_csv("../data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["UserID", "DateTime", "History", "ClickData"])
AllData = pd.concat([AllTrainingData, AllValidationData], ignore_index=True)

ArticlesTrain = pd.read_csv("../data/MINDsmall_train/news.tsv", sep="\t", header=None, names=["NewsID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
ArticlesValidation = pd.read_csv("../data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["NewsID", "Category", "SubCategory", "Title", "Abstract", "URL", "TitleEntities", "AbstractEntities"])
AllArticles = pd.concat([ArticlesTrain, ArticlesValidation], ignore_index=True)

ArticlesTrainWithTime = pd.read_csv("../data/NewsWithTime/small/TrainNewsWithTime.csv")
ArticlesValidationWithTime = pd.read_csv("../data/NewsWithTime/small/DevNewsWithTime.csv")
AllArticlesWithTime = pd.read_csv("../data/NewsWithTime/small/AllNewsWithTime.csv")

PreparationEndTime = time.time()
PreparationTime += PreparationEndTime - PreparationStartTime

In [94]:
#Maybe add something to reduce the amount of data??

In [95]:
if TypeOfRecAlg == 2 or TypeOfRecAlg == 3:
    PreparationStartTime = time.time()
    colab_filter = CollaborativeFiltering.CollaborativeFiltering(AllTrainingData, epochs=3)
    
    colab_filter.initialize()
    PreparationEndTime = time.time()
    PreparationTime += PreparationEndTime - PreparationStartTime

Initializing collaborative filtering...
apply start
explode start
sparse matrix start
Starting ALS using cpu
interaction_sparse.shape:  torch.Size([50000, 33196])
user_embeddings.shape, item_embeddings.shape:  torch.Size([50000, 3]) torch.Size([33196, 3])
training start
Epoch 1/3, Loss: 57.32339859008789
Epoch 2/3, Loss: 55.42607116699219
Epoch 3/3, Loss: 55.368953704833984


In [96]:
if TypeOfRecAlg == 1 or TypeOfRecAlg == 3:
    PreparationStartTime = time.time()
    path_items = "../data/MINDsmall_train/news.tsv"
    path_user_behavior = "../data/MINDsmall_train/behaviors.tsv"
    
    recommender = News_Recommender_CBF.NewsRecommenderCBF(path_items, path_user_behavior)
    PreparationEndTime = time.time()
    PreparationTime += PreparationEndTime - PreparationStartTime

00 ----------> ITEM data loaded successfully: 51282 records!
01 ----------> USER data loaded successfully: 156965 records!
02 ----------> Corpus created: 51282 documents!
03 ----------> TF-IDF matrix created: 51282 documents, 167113 terms!
04 ----------> Category matrix created: 51282 documents, 281 categories!
05 ----------> Combined matrix created, shape: (51282, 167394)


In [97]:
PreparationStartTime = time.time()
PopularityDict = {}
for row in AllData.itertuples(index=False):
    for click in row.ClickData.split(" "):  # split string of clicks
        if click.endswith("-1"):  # Only clicked articles
            ArticleID = click.replace("-1", "")
            PopularityDict[ArticleID] = PopularityDict.get(ArticleID, 0) + 1
PopularityDict = sorted(PopularityDict.items(), key=lambda x: x[1], reverse=True)
PreparationEndTime = time.time()
PreparationTime += PreparationEndTime - PreparationStartTime

In [98]:
PreparationStartTime = time.time()
#Main loop
#Assume we use the past behaviors we have to predict the click behavior on the test set (-1's aka clicked articles)
#We hope our recommendations include these articles
TotalAUCEvalScore = 0
TotalMMREEvalScore = 0
TotalNDCG10EvalScore = 0
TotalNDCG5EvalScore = 0
i=0
amountOfColdStarts = 0
TotalInferenceTime = 0

# Preprocessing before the loop 
ArticlesValidationWithTime = ArticlesValidationWithTime.sort_values('ReleaseDate').reset_index(drop=True)
ArticlesValidationWithTime['ReleaseDate'] = pd.to_datetime(ArticlesValidationWithTime['ReleaseDate'])
ReleaseDates = pd.to_datetime(ArticlesValidationWithTime['ReleaseDate'].values)

# Sample set amount of instances in validation
AllValidationData = AllValidationData.sample(n=10000, random_state=88).reset_index(drop=True)

AllBehaviors = AllValidationData.sort_values('DateTime').reset_index(drop=True)
AllBehaviors['DateTime'] = pd.to_datetime(AllBehaviors['DateTime'])
AllTimes = pd.to_datetime(AllBehaviors['DateTime'].values)
PreparationEndTime = time.time()
PreparationTime += PreparationEndTime - PreparationStartTime

for _, instance in tqdm(AllValidationData.iterrows(), total=len(AllValidationData), desc="Processing Instances"):
    # Start timing inference
    InferenceStartTime = time.time()
    # Get necessary parameters
    UserID = instance['UserID']
    Time = pd.to_datetime(instance['DateTime'])
    cutoff_index = ReleaseDates.searchsorted(Time, side='right')
    AvailableNews = ArticlesValidationWithTime.iloc[:cutoff_index]
    
    
    future_start_idx = AllTimes.searchsorted(Time, side='left')
    FutureBehaviors = AllBehaviors.iloc[future_start_idx:]
    GT = getGroundTruth(FutureBehaviors, UserID, instance['ClickData'])
    
    # skip user if there is no future data for this user
    if len(GT) == 0:
        print("skipped user")
        continue
        
    # Run the selected RecAlg
    if TypeOfRecAlg == 0:
        # PossibleArticles, CurrentTime, GlobalPopularity, TimePenaltyPerHour, TimePenaltyStart
        TopTenArticleRecommendations = MostPopBaseline.RecommendMostPopular(AvailableNews, Time, PopularityDict, TimePenaltyPerHour, TimePenaltyStart, AmountToPredict)
    elif TypeOfRecAlg == 1:
        TopTenArticleRecommendations = recommender.recommend(UserID, AmountToPredict)

    elif TypeOfRecAlg == 2:
        TopTenArticleRecommendations = colab_filter.getRecommended(UserID, k=AmountToPredict)

    elif TypeOfRecAlg == 3:
        PopRec = MostPopBaseline.RecommendMostPopular(AvailableNews, Time, PopularityDict, TimePenaltyPerHour, TimePenaltyStart, -1)
        CFRec = colab_filter.getRecommended(UserID, -1)
        CBRRec = recommender.recommend(UserID, -1)
        TopTenArticleRecommendations = Hybrid.HybridRecommendations(PopRec, CFRec, CBRRec, Weights, AmountToPredict)
    else:
        continue
        
    # For cold start
    if len(TopTenArticleRecommendations) == 0:
        amountOfColdStarts += 1
        TopTenArticleRecommendations = MostPopBaseline.RecommendMostPopular(AvailableNews, Time, PopularityDict, TimePenaltyPerHour, TimePenaltyStart, AmountToPredict)
        
    # End timing inference
    InferenceEndTime = time.time()
    TotalInferenceTime += (InferenceEndTime - InferenceStartTime)
    
    # Calculate evaluation scores
    AUCScore = AUCEval.AUCEval(TopTenArticleRecommendations, GT)
    MMREScore = MMREval.MMREval(TopTenArticleRecommendations, GT)
    NDCG10Score = nDCGEval.nDCG(TopTenArticleRecommendations, GT)
    NDCG5Score = nDCGEval.nDCG(TopTenArticleRecommendations[:5], GT)
    
    # Print the scores for the current user and generation
    # print(f"Generation {i}: User {UserID} - AUC: {AUCScore}, MMRE: {MMREScore}, NDCG: {NDCGScore}")
    
    # Accumulate the total scores
    TotalAUCEvalScore += AUCScore
    TotalMMREEvalScore += MMREScore
    TotalNDCG10EvalScore += NDCG10Score
    TotalNDCG5EvalScore += NDCG5Score
    i+=1


AvgAUCScore = TotalAUCEvalScore/i
AvgMMREScore = TotalMMREEvalScore/i
AvgNDCG10Score = TotalNDCG10EvalScore/i
AvgNDCG5Score = TotalNDCG5EvalScore/i
AvgInferenceTimePerUser = TotalInferenceTime / i


Processing Instances: 100%|██████████| 10000/10000 [1:36:52<00:00,  1.72it/s] 


In [99]:
# Average Evaluation
print(f"Average AUC Score: {AvgAUCScore:.20f}")
print(f"Average MMRE Score: {AvgMMREScore:.20f}")
print(f"Average NDCG10 Score: {AvgNDCG10Score:.20f}")
print(f"Average NDCG5 Score: {AvgNDCG5Score:.20f}")
print(f"Average Inference Time per User: {AvgInferenceTimePerUser:.6f} seconds")
print(f"Preparation Time: {PreparationTime:.4f} seconds")
print(f"Number of Cold Starts: {amountOfColdStarts}")
# Look at the results, and evaluate them with the different evaluation functions

Average AUC Score: 0.12668333333333331447
Average MMRE Score: 0.09139641691601113616
Average NDCG10 Score: 0.15371915444688044605
Average NDCG5 Score: 0.14191051055387990387
Average Inference Time per User: 0.579979 seconds
Preparation Time: 151.6041 seconds
Number of Cold Starts: 0
